In [13]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re

In [40]:
data = pd.read_csv('feedback_dataset.csv')

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       5200 non-null   object
 1   sentiment  5200 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 81.4+ KB


In [43]:
def to_sentiment(sentiment):
  sentiment = sentiment
  if sentiment == 0:
    return 'negative'
  else: 
    return 'positive'
data['sentiment'] = data.sentiment.apply(to_sentiment)

In [44]:
def preProcess_data(text):
   text = text.lower()
   new_text = re.sub('[^a-zA-z0-9\s]','',text)
   new_text = re.sub('rt', '', new_text)
   return new_text

data['text'] = data['text'].apply(preProcess_data)

In [51]:
max_fatures = 2000

tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X, 28) 

Y = pd.get_dummies(data['sentiment']).values

In [52]:
Y

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [53]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20)

In [56]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(128,recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [57]:
batch_size = 512

model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, validation_data=(X_test, Y_test))

Epoch 1/10
9/9 [==============================] - 18s 1s/step - loss: 0.6748 - accuracy: 0.5572 - val_loss: 0.6376 - val_accuracy: 0.6221
Epoch 2/10
9/9 [==============================] - 8s 901ms/step - loss: 0.6350 - accuracy: 0.6308 - val_loss: 0.6112 - val_accuracy: 0.6538
Epoch 3/10
9/9 [==============================] - 9s 1s/step - loss: 0.5989 - accuracy: 0.6726 - val_loss: 0.5704 - val_accuracy: 0.7000
Epoch 4/10
9/9 [==============================] - 9s 983ms/step - loss: 0.5273 - accuracy: 0.7303 - val_loss: 0.4884 - val_accuracy: 0.7625
Epoch 5/10
9/9 [==============================] - 10s 1s/step - loss: 0.4215 - accuracy: 0.8173 - val_loss: 0.4205 - val_accuracy: 0.8058
Epoch 6/10
9/9 [==============================] - 9s 990ms/step - loss: 0.3366 - accuracy: 0.8584 - val_loss: 0.3830 - val_accuracy: 0.8500
Epoch 7/10
9/9 [==============================] - 8s 938ms/step - loss: 0.2665 - accuracy: 0.8993 - val_loss: 0.3687 - val_accuracy: 0.8587
Epoch 8/10
9/9 [===========

In [58]:
# we have saved our model in ‘hdf5’ format
model.save('sentiment.h5')